# Introduction to the Planner

The Planner is one of the fundamental concepts of the Semantic Kernel. It makes use of the collection of plugins that have been registered to the kernel and using AI, will formulate a plan to execute a given ask.

Read more about it [here](https://aka.ms/sk/concepts/planner).

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.0.0-rc3"
#r "nuget: Microsoft.SemanticKernel.Planners.Handlebars, 1.0.0-rc3"

#!import config/Settings.cs
#!import config/Utils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.AI.OpenAI;

var builder = new KernelBuilder();

// Configure AI backend used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, "model-id", azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

var kernel = builder.Build();

### Setting Up Handlebars Planner
Handlebars Planner is located in the `Microsoft.SemanticKernel.Planning.Handlebars` package.

In [ ]:
using Microsoft.SemanticKernel.Planning.Handlebars;

#pragma warning disable SKEXP0060

var planner = new HandlebarsPlanner();

### Providing plugins to the planner
The planner needs to know what plugins are available to it. Here we'll import the `SummarizePlugin` and `WriterPlugin` we have defined on disk.

In [ ]:
var pluginsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "..", "..", "samples", "plugins");

kernel.ImportPluginFromPromptDirectory(Path.Combine(pluginsDirectory, "SummarizePlugin"));
kernel.ImportPluginFromPromptDirectory(Path.Combine(pluginsDirectory, "WriterPlugin"));

Define your ASK. What do you want the Kernel to do?

In [ ]:
#pragma warning disable SKEXP0060

var ask = "Tomorrow is Valentine's day. I need to come up with a few date ideas. My significant other likes poems so write them in the form of a poem.";
var originalPlan = await planner.CreatePlanAsync(kernel, ask);

Console.WriteLine("Original plan:\n");
Console.WriteLine(originalPlan);

As you can see in the above plan, the Planner has taken the user's ask and converted it into a Plan object detailing how the AI would go about solving this task.

It makes use of the plugins that the Kernel has available to it and determines which functions to call in order to fulfill the user's ask.

Let's also define an inline plugin and have it be available to the Planner.
Be sure to give it a function name and plugin name.

In [ ]:
string skPrompt = """
{{$input}}

Rewrite the above in the style of Shakespeare.
""";

var executionSettings = new OpenAIPromptExecutionSettings 
{
    MaxTokens = 2000,
    Temperature = 0.7,
    TopP = 0.5
};

var shakespeareFunction = kernel.CreateFunctionFromPrompt(skPrompt, executionSettings, "Shakespeare");

Let's update our ask using this new plugin.

In [ ]:
#pragma warning disable SKEXP0060

var ask = @"Tomorrow is Valentine's day. I need to come up with a few date ideas.
She likes Shakespeare so write using his style. Write them in the form of a poem.";

var newPlan = await planner.CreatePlanAsync(kernel, ask);

Console.WriteLine("Updated plan:\n");
Console.WriteLine(newPlan);

### Executing the plans

Now that we have different plans, let's try to execute them! The Kernel can execute the plan using RunAsync.

In [ ]:
#pragma warning disable SKEXP0060

var originalPlanResult = originalPlan.Invoke(kernel, new KernelArguments());

Console.WriteLine("Original Plan results:\n");
Console.WriteLine(Utils.WordWrap(originalPlanResult.ToString(), 100));

Now lets execute and print the new plan:

In [ ]:
#pragma warning disable SKEXP0060

var newPlanResult = newPlan.Invoke(kernel, new KernelArguments());

Console.WriteLine("New Plan results:\n");
Console.WriteLine(newPlanResult);